<a href="https://colab.research.google.com/github/StringBottle/kaggle-titanic/blob/main/Kaggle_Titanic_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# About

### Kaggle Competition | Titanic Machine Learning from Disaster 


#### Goal for this Notebook: 

End to end example of predicting survival of Titanic passengers with Google Colaboratory



Data Handling 
* Getting dataset with Kaggle CLI
* Importing Data with Pandas 
* Cleaning Data 
* Submitting predictions with Kaggle CLI
* Mounting Google Drive as a partition in Google Colab
* Using github for file transfer

### 0. Define global variables 

These are *NOT HYPER PARAMETERS* and primarily for code maintenance

In [1]:
KAGGLE_COMPETITION_NAME='titanic'
GDRIVE_ROOT='/content/my_google_drive'
GDRIVE_KAGGLE_TITANIC='/content/my_google_drive/kaggle-titanic'
GDRIVE_KAGGLE_TITANIC_AI_CODE='/content/my_google_drive/kaggle-titanic/kt-ai-code'
RESULTS='/content/my_google_drive/kaggle-titanic/results'
RESULT_CSV='/content/my_google_drive/kaggle-titanic/results/submission.csv'

KAGGLE_CLI_COLAB_DATA='/content/.kaggle/competitions'
KAGGLE_CLI_COLAB_ROOT='/content/.kaggle'

IS_MAX_SCORE_SUBMISSION = True # create submission.csv of maximum result
FINAL = False # indicate the predication using trained models is saved to CSV

### 1. Link google drive to google colaboratory file system 

This mounts the google drive as a file system in the google colaboratory Virtual Machine (VM)

In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
··········


mount my google drive 

In [4]:
!mkdir -p my_google_drive
!google-drive-ocamlfuse my_google_drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


#### Put titanic dataset into google drive 

Create folder for competition data and AI 

In [11]:
!ls $GDRIVE_ROOT

'구조물 영상분석 알고리즘 고도화(190610,초안).hwp'
 2.crack_detection_result.jpg
'BH&YN Daily Routine.desktop'
'Colab Notebooks'
'무인체_성과활용보고서_CTE3-4(시립대)_ver.0.2.hwp'
'제목 없는 그림.desktop'
'제목 없는 스프레드시트.desktop'
 실험준비.desktop
'fix case back.GIF'
'굉장히 funny 한 레크레이션.pptx'
'시립대 파워 거래명세서.hwp'
 KakaoTalk_Moim_PKuy8m3HNGlwjRp5pdvgjcGHaAzpn.jpg
'NIA 2020년 중간결과보고_서울시설공단_200918.pptx'
'NIA 2020년 중간결과보고 발표자료_서울시립대학교_200921.pptx'
'Ⅴ._프로젝트_관리_인력_서울시립대(수정).pptx'
 S1.JPG
 S1.JPG.desktop
'Untitled spreadsheet.desktop'
'UOS_PT member(2016version).dwg'
'Ⅲ. 기술 및 기능_v0.2(취합)_20180822_수정중.pptx'


In [12]:
!mkdir $GRIVE_ROOT/kaggle-titanic

mkdir: cannot create directory ‘/kaggle-titanic’: File exists


check your related drive folder exist or not. 

In [10]:
!ls $GDRIVE_KAGGLE_TITANIC

ls: cannot access '/content/my_google_drive/kaggle-titanic': No such file or directory


#### Link to Kaggle 

Use the Kaggle CLI to get the competition dataset 

Install kaggle

In [7]:
!pip install kaggle

#### Kaggle API Credentials


To use the Kaggle API, sign up for a Kaggle account at https://www.kaggle.com. Then go to the 'Account' tab of your user profile (https://www.kaggle.com//account) and select 'Create API Token'. This will trigger the download of kaggle.json, a file containing your API credentials.

Place this file on your Google Drive anywhere.

With the next snippet you download your credentials to Colab and you can start using Kaggle API

In [9]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


##### confirm transfer of kaggle api keys to google colab

In [11]:
!Is $KAGGLE_CLI_COLAB_ROOT/kaggle.json

/bin/bash: Is: command not found
